<img src="./101_ESSENCE.png" width=700px>

### Reflection pattern

## input -> function(input) -> output -> function(output) -> output2

<img src="./100_INPUT_OUTPUT.png" width=700px>

Each LLM call is stateless so the LLM sees it for the first time. The more information (context) it has the better will be its response.

We can use this pattern to implement a reflection pattern in stages or we could have gathered the information in another way and passed it in a single request as the first time.




<img src="./images/reflection.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

In [ ]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown
from openai import OpenAI
# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

# client = Groq() alternative way to use Groq

In [ ]:
def get_llm_client(llm_choice):
    if llm_choice == "GROQ":
        client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=os.environ.get("GROQ_API_KEY"),
        )
        return client
    elif llm_choice == "OPENAI":
        load_dotenv()  # load environment variables from .env fil
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        return client
    else:
        raise ValueError("Invalid LLM choice. Please choose 'GROQ' or 'OPENAI'.")

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# LLM_CHOICE = "OPENAI"
LLM_CHOICE = "GROQ"

if OPENAI_API_KEY:
    print(f"OPENAI_API_KEY exists and begins {OPENAI_API_KEY[:14]}...")
else:
    print("OPENAI_API_KEY not set")

if GROQ_API_KEY:
    print(f"GROQ_API_KEY exists and begins {GROQ_API_KEY[:14]}...")
else:
    print("GROQ_API_KEY not set")


client = get_llm_client(LLM_CHOICE)
if LLM_CHOICE == "GROQ":
    MODEL = "llama-3.3-70b-versatile"
else:
    MODEL = "gpt-4o-mini"

print(f"LLM_CHOICE: {LLM_CHOICE} - MODEL: {MODEL}")

In [ ]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [ ]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the an AI Agent that uses the ReAct thought-observer architecture"
    }
)

Let's generate the first version of the essay.

In [6]:
generated_code = client.chat.completions.create(
    messages=generation_chat_history,
    model=MODEL, 
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": generated_code
    }
)

In [7]:
# display_markdown(generated_code, raw=True)
with open("200_code.md", "w") as f:
    f.write(generated_code)

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt.

In [8]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are an experienced Python reviewer. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [9]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": generated_code
    }
)

Now, let's generate a critique to the Python code.

In [10]:
# critique = client.chat.completions.create(
#     messages=reflection_chat_history,
#     model="llama3-70b-8192"
# ).choices[0].message.content

critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model=MODEL, 
).choices[0].message.content

In [11]:
# display_markdown(critique, raw=True)
with open("200_critique.md", "w") as f:
    f.write(critique)

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [12]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [13]:
# essay = client.chat.completions.create(
#     messages=generation_chat_history,
#     model="llama3-70b-8192"
# ).choices[0].message.content

essay = client.chat.completions.create(
    messages=generation_chat_history,
    model=MODEL, 
).choices[0].message.content

In [14]:
# display_markdown(essay, raw=True)
with open("200_final.md", "w") as f:
    f.write(essay)